# Lesson 5 project: Histograms and Monte Carlo on GPUs

In [ ]:
import math
import copy

import h5py
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
import numba as nb
import numba.cuda

import uproot
import awkward as ak
from hist import Hist

The three exercises below are independent: you can start with whichever one you want.

<br><br><br>

## Exercise 1: make a CUDA kernel with Numba

We'll use the non-ragged data from [lesson-2-project.ipynb](lesson-2-project.ipynb) by casting each array from an HDF5 file as CuPy arrays.

In [ ]:
dataset_hdf5 = h5py.File("data/SMHiggsToZZTo4L.h5")
e1_pt = cp.asarray(dataset_hdf5["ee_mumu"]["e1"]["pt"])
e1_phi = cp.asarray(dataset_hdf5["ee_mumu"]["e1"]["phi"])
e1_eta = cp.asarray(dataset_hdf5["ee_mumu"]["e1"]["eta"])
e2_pt = cp.asarray(dataset_hdf5["ee_mumu"]["e2"]["pt"])
e2_phi = cp.asarray(dataset_hdf5["ee_mumu"]["e2"]["phi"])
e2_eta = cp.asarray(dataset_hdf5["ee_mumu"]["e2"]["eta"])

We can use CuPy's array interface to compute the mass with

$$\sqrt{2\,{p_T}_1\,{p_T}_2\left(\cosh(\eta_1 - \eta_2) - \cos(\phi_1 - \phi_2)\right)}$$

In [ ]:
z_mass = ...

Now implement the same as a single kernel with Numba.

In [ ]:
@nb.cuda.jit
def compute_z_mass(...):
    ...

#

In [ ]:
#run compute_z_mass and store the computed Z mass in a variable called z_mass_numba
....

compute_z_mass.....

And finally make a plot of each. Here is a histogram definition to get you started

In [ ]:
fig, ax = plt.subplots()

z_mass_numba = z_mass
Hist.new.Reg(120, 0, 120, name="e+e- mass (GeV)").Double().fill(z_mass.get()).plot()

# are the two histograms correctly overlaid? Eg, 
None

In [ ]:
fig, ax = plt.subplots()

z_mass_numba = z_mass
Hist.new.Reg(120, 0, 120, name="e+e- mass (GeV)").Double().fill(z_mass_numba.get()).plot()

# are the two histograms the same??
None

## Exercise 2: fill a histogram on the GPU

This time, you'll fill the histogram on the GPU and only copy the filled bin contents from GPU to RAM.

The dataset is larger, 10 million dimuon masses. Pretend that the histogram code you're writing will be applied to data that was computed on the GPU, to avoid a costly data transfer before reducing it to a small set of bin contents.

In [ ]:
with uproot.open("data/dimuon_mass.root:tree/mass") as branch:
    dimuon_mass = cp.asarray(branch.array(library="np"))

The histogram will have 1200 bins and cover a range from 0 to 120 GeV, with the overflows going in the nearest edge bin. Thus, the mass-to-bin calculation is simple:

```python
bin_index = int(mass * 10)
if bin_index < 0:
    bin_index = 0
elif bin_index >= 1200:
    bin_index = 1199
```

and the plot should look like

![](img/lesson-5-exercise-2-expectation.png)

The histogram bins is another array that you'll pass to the Numba-based kernel function.

In [ ]:
bin_values = cp.zeros(1200, dtype=cp.uint32)

CUDA supplies some atomic operations and Numba makes them available as `nb.cuda.atomic.*` ([see documentation](https://numba.readthedocs.io/en/stable/cuda/intrinsics.html)). Which one(s) would be useful here?

Note that atomic operations are only defined for a limited set of numeric data types (`dtype`). If you need to define `bin_values` as an unsigned integer, go ahead.

In [ ]:
@nb.cuda.jit
def fill_histogram(...):
        ...

...

fill_histogram ... 

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))

h = Hist.new.Reg(1200, 0, 120, label="dimuon mass (GeV)").Double()
h.values()[:] = bin_values.get()
h.plot(ax=ax)

ax.set_xlim(0, 120)
ax.set_yscale("log")

None

<br><br><br>

## Exercise 3: use Monte Carlo to calculate π

All of the examples we've seen so far use the GPU to analyze data. What if you want to generate Monte Carlo samples?

Random numbers need to be used carefully with parallel processing, since it's easy to accidentally generate the same (or correlated) random numbers on different processors, which invalidates the result in hard-to-discover ways. This is because "random numbers" really means "arbitrary numbers", since a GPU, like any other computational device, is deterministic. A sequence of "random numbers" is a sequence that would be unsurprising if we were expecting a uniformly distributed, statistically independent set.

For instance, if we copied a NumPy random number generator to two processes running in parallel,

In [ ]:
rng1 = np.random.default_rng()
rng2 = copy.deepcopy(rng1)

both processes would generate the same data.

In [ ]:
rng1.integers(0, 100, 15)

In [ ]:
rng2.integers(0, 100, 15)

To prevent this, [np.random.Generator](https://numpy.org/doc/stable/reference/random/generator.html) has a [spawn](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.spawn.html) method, which makes a set of generators that are guaranteed independent and uncorrelated.

In [ ]:
rng = np.random.default_rng()

rng1, rng2 = rng.spawn(2)

In [ ]:
rng1.integers(0, 100, 15)

In [ ]:
rng2.integers(0, 100, 15)

(The correlation coefficient is about 0.001 for off-diagonal elements in a set of a million Gaussian-distributed values.)

In [ ]:
np.corrcoef(rng1.normal(0, 1, 1000000), rng2.normal(0, 1, 1000000))

For a GPU kernel, we need to spawn as many random number generators as we have threads. For a massively parallel process, that can be a lot of seeds!

In Numba, the functions that spawn and use these generators are called `nb.cuda.random.*xoroshiro128p*` ([see documentation](https://numba.readthedocs.io/en/stable/cuda/random.html)).

In [ ]:
from numba.cuda.random import create_xoroshiro128p_states
from numba.cuda.random import xoroshiro128p_uniform_float32

This generates 10000 generators (outside of the kernel) and then runs each generator for 1000 steps (inside the kernel) to get 10000 × 1000 random floating point numbers.

In [ ]:
@nb.cuda.jit
def generate_uniform(rng_states, out):
    thread_idx = nb.cuda.grid(1)
    for j in range(1000):
        out[thread_idx, j] = xoroshiro128p_uniform_float32(rng_states, thread_idx)

out = cp.empty((10000, 1000), dtype=np.float32)

threads_per_block = 1024
blocks_per_grid = int(np.ceil(len(out) / 1024))

rng_states = create_xoroshiro128p_states(threads_per_block * blocks_per_grid, seed=12345)

generate_uniform[blocks_per_grid, threads_per_block](rng_states, out)

out

For performance, there's a balance between having enough blocks to keep the whole GPU busy and having to spawn such a large number of generators (outside the kernel) that the spawning process takes more time than the GPU kernel.

In this exercise, you need to sample random number pairs $x \in (-1, 1)$, $y \in (-1, 1)$ with a uniform distribution to compute the area of a circle with radius $r = 1$. Then use the area $A = \pi r^2$ formula to derive $\pi$.

* How does the accuracy depend on number of sampled pairs?
* What about the computation time?